In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install git+https://github.com/davidbau/baukit > /dev/null

## Using baukit show()

`show()` is similar to notebook's `display()`, but it is gives you more control over layout and rendering of the HTML.

To start, you can use it just like `print()` or `display()`.

In [ ]:
from baukit import show

show('hello', 'world')

## Easy layouts by nesting arrays

`show()` makes it easy to produce layouts of data by arranging it in nested arrays.  At the top level, data is arranged in rows; then at the second level, data is arranged in columns; then at the third level, it is split into rows again, and so on.

(Technical detail: the HTML it produces uses [CSS flexbox layouts](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Flexible_Box_Layout/Basic_Concepts_of_Flexbox), alternating row and column flex containers for each level of nesting.)

In [ ]:
show(['first row ' * 30,
      ['second row ' * 10, 'second row second item ' * 5],
      ['third row ' * 10, ['third row second item ' * 5, ['some more', 'data', 'here']]]])

## A simple grid layout example

A very common pattern is to show data in a grid.  Here is a three-level nested piece of retangular data:

In [ ]:
show([[['hello', [i, j]] for j in range(1, 8)] for i in range(1, 6)])

## Showing images

Unlike `display()`, `show()` knows how to directly render PIL images and matplotlib figures and other types of data.  It also provides more powerful layout in HTML and gives you much more control over CSS formatting.  

In [ ]:
from PIL import Image
from numpy.random import randint

show([[[f'image {i}',
        Image.fromarray(randint(0,255,(128,128,3),dtype='uint8'))]
       for i in range(10)]])


## Controlling CSS

The style of anything emitted by show can be controlled by using `show.style()` to inject CSS properties.  `show.style` will alter the CSS style of the next item rendered by `show()`.

In [ ]:
show([show.style(background='pink', font='italic 24px serif'), 'A demonstration of CSS control',
      [show.style(background='skyblue'), 2, show.style(background='yellow', flex=2), 3],
      show.style(background='lightgreen'), 4])

## Tight columns and Wrapped rows

A few useful `show.style` instances are defined as constants.  For example, `show.style(display='inline-flex')` is also called `show.TIGHT`, because it provides a tight layout of rows that are sized to fit the content instead of making the flexbox expand to fill its container.  Similaly `show.WRAP` makes a row that packs the data to the left and wraps it (like the way text flows), instad of as spaced-out columns.

In [ ]:
show(show.TIGHT,
     [[[show.style(font='italic 24px serif'), 'lots of data',
        show.style(background='gainsboro'),
     show.WRAP, [f'({a},{b})' for a in range(i) for b in range(j)]]
       for j in range(1, 5)] for i in range(1, 3)])

## Showing widgets

baukit also contains a bunch of widgets like `Range()` and `Numberbox()` that can be shown and laid out:

In [ ]:
from baukit import Range, Numberbox

nb = Numberbox()
ra = Range()

show([[nb, show.style(flex=5), ra]])

## Getting and setting widget values

Widgets are live data-bound objects.  So if you change a value in the widget, it will display that value right away.  Also, if the user enters some input, it will be reflected in the python value right away.  To see that effect, run the following cells while interacting with the widgets above:

In [ ]:
import random
nb.value = random.randrange(100)

In [ ]:
print(nb.value)
print(ra.value)

## Binding live widget properties

Widget properties can be bound together - if you refer to a widget property using the `widget.prop()` method, it will return a live property object that can be bound to another property.

(Also notice that if you show the same widget instance multiple times, all the rendered views will in sync.)

In [ ]:
nb2 = Numberbox()
ra2 = Range(max=1, step=0.001)

ra2.value = nb2.prop('value')

show([[nb2, show.style(flex=6), ra2]])
show(ra2)


## Using PlotWidget

Matplotlib plots can be displayed within HTML by showing the matplotlib figure.

`PlotWidget` is a widget that manages the matplotlib figure and allows you to create a parameterized plot by writing a plotting function.  Any arguments in your plotting function become parameters of the PlotWidget that you can adjust.




In [ ]:
import torch
from baukit import PlotWidget

def myplot(plt, frequency=1.0, amplitude=1.0):
    [ax] = plt.axes
    ax.clear()
    ax.set_title(f'Sine wave of frequency {frequency}, amplitude {amplitude}')
    x = torch.linspace(0, 20, 300)
    y = (frequency * x).sin()
    ax.plot(x, amplitude * y)
    ax.set_ylim(-5, 5)

show([[PlotWidget(myplot, format='svg', figsize=(5,3.5)), PlotWidget(myplot, frequency=2, amplitude=3, format='svg', figsize=(5,3.5))]])

## Making interactive matplotlib plots with PlotWidget

`PlotWidget` properties can be bound together just like any other widget properties, so it is easy to define an interactive plot interface.

In [ ]:

freq_ra = Range(min=0.1, max=5.0, step=0.01, value=1.0)
freq_nb = Numberbox(freq_ra.prop('value'))
amp_ra = Range(min=0.1, max=5.0, step=0.01, value=1.0)
amp_nb = Numberbox(amp_ra.prop('value'))
pw = PlotWidget(myplot, frequency=freq_ra.prop('value'), amplitude=amp_ra.prop('value'))
show(show.TIGHT, [[pw],
                  ['frequency', show.style(flex=10), freq_ra, freq_nb],
                  ['amplitude', show.style(flex=10), amp_ra, amp_nb]])